## Computational Finance Assignment Three

### Student Code: VP5008RP

In [1]:
import numpy as np;
import matplotlib.pyplot as plt
import math as m
import pandas as pd

### Valuing European put option

In [2]:
class Binomial_tree:
    """
    Binomial Tree method for computing the value of European put options. The class is instantiated
    with some predefined variables of S0=5, E=10, r=0.12 and sigma = 0.5; The class compute the value based on 
    two methods of computing the upward and downward factors once specify in the tree_methods method of the 
    class object. "UDP1" uses the Cox-Ross-Rubenstein method whereas the "UDP2" uses the lognormal formula for 
    the upward and downward factors with each associated probability "p".
    
    """
    
    def __init__(self, E, S0, r, sigma):
        self.E = E
        self.S0 = S0
        self.r = r
        self.sigma = sigma
    
    def tree_methods(self, name, T, M):
        dt = T/float(M)
        
        if name == "UDP1":
            u = m.exp(self.sigma*m.sqrt(dt))
            d =  m.exp(-self.sigma*m.sqrt(dt))
            p = (m.exp(self.r*dt)-d)/(u-d)
            
            assetAtExpiry = (self.S0*d**np.arange(M,-1,-1))*(u**np.arange(0,M+1, 1))
        
            optionValue_Put = np.maximum(self.E-assetAtExpiry, 0)
        
            for i in range(int(M)-1,-1,-1):
            
                optionValue_Put = m.exp(-self.r*dt)*(p*optionValue_Put[range(1, i+2)] + (1-p)*optionValue_Put[range(0,i+1)])
    
            return optionValue_Put[0]
            
        
        elif name == "UDP2":
            
            u = m.exp(self.sigma*m.sqrt(dt) + (self.r-(self.sigma**2)/2)*dt)
            d = m.exp(-self.sigma*m.sqrt(dt) + (self.r-(self.sigma**2)/2)*dt)
            p = 0.5
            
            assetAtExpiry = (self.S0*d**np.arange(M,-1,-1))*(u**np.arange(0,M+1, 1))
        
            optionValue_Put = np.maximum(self.E-assetAtExpiry, 0)
        
            for i in range(int(M)-1,-1,-1):
            
                optionValue_Put = m.exp(-self.r*dt)*(p*optionValue_Put[range(1, i+2)] + (1-p)*optionValue_Put[range(0,i+1)])
    
            return optionValue_Put[0]

tree = Binomial_tree(10, 5, 0.12, 0.5)

#### Initializing the values for M and T

In [3]:
### Values for M upward-steps along the tree and T different expiry periods.
M = [16, 32, 64, 128, 256]
T = [0.25, 0.50, 0.75, 1.0]

### get_values function to do calculation for the various M and T values 

In [4]:
def get_values(T, M, method):
    vals = []
    for year in T:
        for steps in M:
            vals.append(tree.tree_methods(method, year, steps))
    values = np.array(vals).reshape(len(T),len(M))
    Data_Frame = pd.DataFrame(values, index = ["T=0.25", "T=0.50", "T=0.75", "T =1.0"], 
                              columns = ["M=16", "M=32", "M=64", "M=128", "M=256"])
    return Data_Frame

### UDP1 Method

#### Calculating the values for each T using the different values for M

In [5]:
get_values(T, M, "UDP1")

,M=16,M=32,M=64,M=128,M=256
T=0.25,4.705996,4.706201,4.706337,4.706516,4.706541
T=0.50,4.447169,4.451853,4.451639,4.451941,4.452525
T=0.75,4.239448,4.246108,4.246108,4.245062,4.246187
T =1.0,4.069143,4.066454,4.073420,4.072181,4.072408


- UDP1 have values closely related with the output in the given table. The table seem to have approximated values for the UDP1 computation

### UDP2: Lognormal  Method

In [6]:
get_values(T, M, "UDP2")

,M=16,M=32,M=64,M=128,M=256
T=0.25,4.706095,4.706235,4.706370,4.706524,4.706551
T=0.50,4.447134,4.452012,4.451592,4.452069,4.452511
T=0.75,4.240646,4.246433,4.246370,4.245407,4.246065
T =1.0,4.069255,4.065847,4.073615,4.071863,4.072777


- The UDP2 method for computing the bionomial tree have the same approximations for the put options as the true Black-Scholes values